<a href="https://colab.research.google.com/github/kothawadegs/Deep-Learning/blob/EVA_4.0%2FS4/MNIST_99_40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.11),
            nn.Conv2d(16, 32, 3, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.11),
            nn.Conv2d(32, 10, 1, stride=1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.1)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(10, 16, 3, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.11),
            nn.Conv2d(16, 16, 3, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, stride=1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(16),
           # nn.MaxPool2d(2, 2),
            nn.Dropout(0.11)
        )
        
        self.conv3 = nn.Sequential(
            #nn.Conv2d(32, 10, 3, padding=0),
            #nn.ReLU(),
            nn.AvgPool2d(6),
            nn.Conv2d(16, 10, 1, padding=0),
            
            #nn.ReLU(),
            #nn.BatchNorm2d(64),
            #nn.AvgPool2d(7),
            #nn.Dropout(0.25)
        )
        
        #self.fc = nn.Sequential(
            #nn.Linear(32, 10)
        #)
                
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x = x.view(x.size(0), -1)
        #x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [3]:
!pip install torchsummary

In [4]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Model().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,640
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
           Dropout-8           [-1, 32, 24, 24]               0
            Conv2d-9           [-1, 10, 24, 24]             330
             ReLU-10           [-1, 10, 24, 24]               0
      BatchNorm2d-11           [-1, 10, 24, 24]              20
        MaxPool2d-12           [-1, 10, 12, 12]               0
          Dropout-13           [-1, 10, 12, 12]               0
           Conv2d-14           [-1, 16,

In [5]:
torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
model = Model().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.7)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.047714173793792725 batch_id=1:   0%|          | 2/1875 [00:00<02:22, 13.11it/s]


Test set: Average loss: 0.0499, Accuracy: 9856/10000 (98.56%)



loss=0.014851056039333344 batch_id=1:   0%|          | 2/1875 [00:00<02:23, 13.05it/s]


Test set: Average loss: 0.0366, Accuracy: 9885/10000 (98.85%)



loss=0.04174429178237915 batch_id=1:   0%|          | 2/1875 [00:00<02:20, 13.37it/s]

...


Test set: Average loss: 0.0218, Accuracy: 9927/10000 (99.27%)



loss=0.024913804605603218 batch_id=1:   0%|          | 2/1875 [00:00<02:15, 13.83it/s]


Test set: Average loss: 0.0211, Accuracy: 9931/10000 (99.31%)



loss=0.07453359663486481 batch_id=1874: 100%|██████████| 1875/1875 [02:22<00:00, 13.12it/s]



Test set: Average loss: 0.0201, Accuracy: 9941/10000 (99.41%)

